# Pre-condition: Setup Environment, download and install Lib

In [ ]:
!.venv/bin/activate 
%pip install selenium
%pip install Pillow
%pip install pandas
%pip install webdriver-manager


# Ý tưởng thực hiện
- Tạo ra 1 `model` để từ trạng thái của game đưa ra quyết định (nhảy/không nhảy). 
- Lựa chọn các trạng thái game nào: `speed` (speed hiện tại của màn chơi), `distance` (khoảng cách từ khủng long tới cây xương rồng gần nhất), `size` (kích thước của cây xương rồng). 
- Tại sao lại lựa chọn những input này?

# Lựa chọn model
- Mở rộng từ phiên bản bot hardcode.
Như đã thực hiện ở phần 1, chúng ta có thể tạo ra 1 phiên bản hardcode chỉ từ việc check xem 1 vùng trước mặt của khủng long có xương rồng hay không. Điều này cho thấy ta có thể xây dựng model không mấy phức tạp và vẫn có thể giải quyết được bài toán.

- Bài toán có thể giải quyết được với nhiều model, ví dụ logistic, decision tree,… 
- Tuy nhiên lựa chọn Neural network để giải quyết bài toán này. Lý do lựa chọn Neural network là bởi:
  - Tôi mới tìm hiểu Neural network, thấy khá thú vị và rộng nên muốn tìm hiểu sâu vào NN trước đã.
  - Sử dụng Neural network thì có thể giải quyết cho các bài toán khác sau này( có thể làm tiếp cho 1 game nào đó).
  - Neural network phổ biến đại trà.

Ok. Vậy chúng ta sẽ lựa chọn các hyper-param như nào cho Neural network của mình? Tôi lựa chọn size 3-3-1 (1 Hidden layer với 3 nodes), lý do bởi vì tôi nghĩ 1 cấu trúc đơn giản như vậy cũng có thể giải quyết bài toán. Output của Neural network sẽ là 1 giá trị action_value, nếu action_value lớn hơn 1 threshold nào đó thì sẽ quyết định sẽ nhảy, còn lại thì không.

# Code phần Neural network
Tạo được 1 model NN, có cấu trúc [3,3,1] với input đầu vào là input_set = [distance, speed, size], và cho ra output là hành động. Phần này được tôi tạo riêng ở file trex_nn.py.


In [ ]:
# Trước hết chúng ta import các package cần thiết. Ở đây import numpy để thực hiện tính toán.
import numpy as np

##### Chúng ta define 2 hàm activation function trong NN là relu và sigmoid

In [ ]:
def relu(array):
    return np.maximum(array, 0.)

def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s


##### Chúng ta cần 1 hàm initialize_parameters để tạo ra các giá trị random ban đầu cho NN. Input đầu vào sẽ là kích thước của NN. Chú ý, chúng ta sẽ nhân các weight và bias với các hệ số riêng, lý do là để scale các giá trị lại với nhau để model hội tụ nhanh hơn.

In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.random.randn(n_h, 1) * 0.5
    W2 = np.random.randn(n_y, n_h) * 1.5
    b2 = np.random.randn(n_y, 1) * 0.15
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters


##### Có 1 hàm nhỏ ta cũng phải xài là re_shape_X nhằm reshape lại input. Lý do là input chúng ta có dạng ma trận ngang input_set = [distance, speed, size], cần chỉnh sửa đôi chút về dạng ma trận dọc.

In [ ]:
def re_shape_X(X, n_x):
    return np.reshape(X, (n_x, 1))


##### Chúng ta dùng hàm tRex_model để tính toán output thông qua input là bộ input X, parameters. Ở đây, parameters là 1 dictionary chứa các parameters. Activation function được sử dụng là sigmoid.

In [ ]:
def tRex_model(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)

    Returns:
    A2 -- The sigmoid output of the second activation
    """
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    return A2

##### Và cuối cùng là hàm wrap_model để áp dụng model vào để chơi

In [ ]:
def from_model_to_action(value, threshold=0.6):
    if value > threshold:
        "JUMP"


def wrap_model(X, parameters, n_x):
    X_adj = re_shape_X(X, n_x)
    action_value = tRex_model(X_adj, parameters).item()
    from_model_to_action(action_value, threshold=0.6)

##### Nhận diện vật thể

##### Test thử model với param đã training

In [5]:
import trex_nn
import SeleniumHelper
import time
import numpy as np

clever_params = {
    'b2': np.array([[0.27304736]]),
    'W2': np.array([[0.91572382, -0.29862268,  0.30955728]]),
    'W1': np.array([[-0.02062025,  0.00016742,  0.00381535],
                    [0.00226537,  0.01325698,  0.02389935],
                    [0.02300561,  0.01351209,  0.00588823]]),
    'b1': np.array([[-0.73119972],
                    [-0.05157346],
                    [-0.00290758]])
}

dinoPlayer = SeleniumHelper.GameDino()
dinoPlayer.press_up()
while True:
    time.sleep(0.01)

    speed = dinoPlayer.get_current_speed()
    distance = dinoPlayer.get_distance_obstacles()
    size = dinoPlayer.get_size_of_obstacle()

    input_set = [distance, speed, size]

    if trex_nn.wrap_model(input_set, clever_params, len(input_set)) == 'JUMP_UP':
        dinoPlayer.wrap_press_up()

KeyboardInterrupt: 